# QT-Assignment-03
by Guillaume IDAME KORALAGE

# Libraries
Let's import libraries we will need first.

In [ ]:
import random

## Questions

The goal of this exercise is to get a flavor of order book modeling. You will simulate a simple particle model, close to the Bak et al. model and analyze the dynamics of mid-point price and bid-ask spread.

In [2]:
def simulate_diffusion(N, M, num_steps, p, print_interval=100):
    if N % 2 != 0:
        raise ValueError("N must be an even number for equal numbers of A and B particles")

    half_N = N // 2
    half_M = M // 2

    # Initialize lists for A and B particle positions with the desired intervals
    a_particle_positions = random.sample(range(half_M, M), half_N)
    b_particle_positions = random.sample(range(half_M), half_N)

    # Create a dictionary to keep track of particle counts at each lattice position
    particle_counts = {}

    # Initialize the particle counts based on initial positions
    for pos in a_particle_positions:
        particle_counts[pos] = ('A', particle_counts.get(pos, 0) + 1)
    for pos in b_particle_positions:
        particle_counts[pos] = ('B', particle_counts.get(pos, 0) + 1)

    mp_evol = []

    # Perform the diffusion simulation for num_steps
    for step in range(num_steps):
        # Shuffle the order in which particles move for both A and B
        random.shuffle(a_particle_positions)
        random.shuffle(b_particle_positions)


        most_right_b = max(b_particle_positions) if b_particle_positions else -1
        most_left_a = min(a_particle_positions) if a_particle_positions else -1
        midpoint  = 0.5*(most_right_b + most_left_a)
        mp_evol.append(midpoint)

        # Randomly select either A or B particle
        chosen_particle_type = random.choice(['A', 'B'])

        if chosen_particle_type == 'A':
            for i in range(len(a_particle_positions)):
                move = random.choices([-1, 0, 1], weights=[p, 1 - 2*p, p])[0]
                current_position = a_particle_positions[i]

                # Calculate the new position after the move without periodic boundary conditions
                new_position = current_position + move

                # Ensure the new position is within the lattice boundaries
                new_position = max(0, min(new_position, M - 1))

                #print('A',current_position, new_position, particle_counts.get(new_position, ('', 0))[0] )

                # Check if the new position is not already occupied by another A particle
                if particle_counts.get(new_position, ('', 0))[0] != 'A' or particle_counts.get(new_position, ('', 0))[1] == 0:
                    # Update the lattice position for the current A particle
                    a_particle_positions[i] = new_position
                    # Update particle_counts by decrementing the count at the old position
                    old_count = particle_counts.get(current_position, ('', 0))[1]
                    particle_counts[current_position] = ('A', old_count - 1)
                    # Update particle_counts by incrementing the count at the new position
                    new_count = particle_counts.get(new_position, ('', 0))[1]
                    particle_counts[new_position] = ('A', new_count + 1)

        else:
            for i in range(len(b_particle_positions)):
                move = random.choices([-1, 0, 1], weights=[p, 1 - 2*p, p])[0]
                current_position = b_particle_positions[i]

                # Calculate the new position after the move without periodic boundary conditions
                new_position = current_position + move

                # Ensure the new position is within the lattice boundaries
                new_position = max(0, min(new_position, M - 1))

                #print('B',current_position, new_position, particle_counts.get(new_position, ('', 0))[0] )

                # Check if the new position is not already occupied by another B particle
                if particle_counts.get(new_position, ('', 0))[0] != 'B' or particle_counts.get(new_position, ('', 0))[1] == 0:
                    # Update the lattice position for the current B particle
                    b_particle_positions[i] = new_position
                    # Update particle_counts by decrementing the count at the old position
                    old_count = particle_counts.get(current_position, ('', 0))[1]
                    particle_counts[current_position] = ('B', old_count - 1)
                    # Update particle_counts by incrementing the count at the new position
                    new_count = particle_counts.get(new_position, ('', 0))[1]
                    particle_counts[new_position] = ('B', new_count + 1)

        # Check for annihilation (A and B particles on the same site)
        a_positions_set = set(a_particle_positions)
        b_positions_set = set(b_particle_positions)
        common_positions = a_positions_set.intersection(b_positions_set)

        for pos in common_positions:
            a_count = a_particle_positions.count(pos)
            b_count = b_particle_positions.count(pos)

            if a_count > 0 and b_count > 0:
                # Annihilation occurs when there's at least one A and one B particle
                a_particle_positions.remove(pos)
                b_particle_positions.remove(pos)
                # Update particle_counts by removing the entry for the annihilated position
                del particle_counts[pos]
                #print("Trade at",pos)
                # insert new particles to avoid depletion
                k=0
                while(particle_counts.get(k, ('', 0))[1]>0):
                  k = k + 1
                particle_counts[k] = ('B',1)
                b_particle_positions.append(k)

                k=M-1
                while(particle_counts.get(k, ('', 0))[1]>0):
                  k = k - 1
                particle_counts[k] = ('A',1)
                a_particle_positions.append(k)


    #print(a_particle_positions)
    #print(b_particle_positions)
    return mp_evol

In [ ]:
# Example usage:
N = 80 # Total number of particles (half will be A and half will be B)
M = 200  # Size of the 1D lattice
num_steps = 100000  # Number of simulation steps
p = 0.3  # Probability of left/right movement

mp_evol = simulate_diffusion(N, M, num_steps, p, print_interval=1)

#### Q1. Consider the following particle model. What are the differences with respect to the model of Per Bak discussed?

#### Q2. Plot the evolution of the midprice (mp_evol). What do you notice ?

#### Q3. Calculate the variogram and signature plot discussed in the first lecture.

#### Q4. Estimate the Hurst exponent.

#### Q5. Modify the code, such that it also outputs the evolution of the spread. Plot the timeseries. What do you notice?

#### Q6. Plot also the histogram.